In [1]:
using CSV
using Dates
using DataFrames
using Dates

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
pct_icu = 0.3;

In [4]:
rawdata = DataFrame(CSV.File("../rawdata/HospitalAdmissionsAverage_20210121.csv"));

In [5]:
get_hospital_id(h) = h |> hash |> string |> (x -> "h" * x[end-7:end]);

In [6]:
data = stack(rawdata, Not("  day"))
select!(data,
    "  day" => ByRow(d -> Date(d, dateformat"m/d/yyyy")) => :date,
    :variable => ByRow(h -> h[1:end-11]) => :hospital,
    :variable => ByRow(h -> get_hospital_id(h[1:end-11])) => :hospital_id,
    :value => :admissions_allbeds,
    :value => ByRow(x -> x * (1-pct_icu)) => :admissions_acute,
    :value => ByRow(x -> x * pct_icu) => :admissions_icu,
);

In [7]:
sort!(data, [:hospital, :date])

,date,hospital,hospital_id,admissions_allbeds,admissions_acute,admissions_icu
,Date,String,String,Float64,Float64,Float64
1,2021-01-01,Anne Arundel Medical Center,h39740093,13.3226,9.32581,3.99677
2,2021-01-02,Anne Arundel Medical Center,h39740093,13.6452,9.55161,4.09355
3,2021-01-03,Anne Arundel Medical Center,h39740093,15.1935,10.6355,4.55806
4,2021-01-04,Anne Arundel Medical Center,h39740093,13.871,9.70968,4.16129
5,2021-01-05,Anne Arundel Medical Center,h39740093,12.8387,8.9871,3.85161
6,2021-01-06,Anne Arundel Medical Center,h39740093,12.3226,8.62581,3.69677
7,2021-01-07,Anne Arundel Medical Center,h39740093,12.9677,9.07742,3.89032
8,2021-01-08,Anne Arundel Medical Center,h39740093,13.1613,9.2129,3.94839
9,2021-01-09,Anne Arundel Medical Center,h39740093,12.3548,8.64839,3.70645


In [8]:
data |> CSV.write("../data/data_forecast.csv")

"../data/data_forecast.csv"